In [88]:
%serialconnect to --port=/dev/ttyUSB0 --baud=115200

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [89]:
import machine
import neopixel
import utime

In [90]:
help(neopixel)

object <module 'neopixel'> is of type module
  NeoPixel -- <class 'NeoPixel'>
  __name__ -- neopixel
  neopixel_write -- <function>


# Test des Neopixels
attention: si le 4iem octet est fort, la couleur tends vers le blanc

In [152]:
Leds=neopixel.NeoPixel(machine.Pin(4),5,bpp=4)

In [180]:
Leds[0]=(100,100,100,100)

In [214]:
Leds[1]=(100,00,00,0)

In [213]:
Leds[2]=(100,0,00,200)

In [198]:
Leds[0]=(0,0,0,0)
Leds[1]=(0,0,0,0)
Leds[2]=(0,0,0,0)

In [215]:
Leds.write()

In [9]:
def blink(leds,pos,color=(100,100,100,100)):
    on=color
    off=(0,0,0)
    utime.sleep_ms(500)
    leds[pos]=color
    leds.write()
    utime.sleep_ms(500)
    leds[pos]=off
    leds.write()

In [33]:
while 1:
    blink(Leds,1)
    blink(Leds,0)


.......

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 3, in <module>
  File "<stdin>", line 7, in blink
KeyboardInterrupt: 


# Connection au réseau wifi

In [92]:
import network

In [93]:
sta_if = network.WLAN(network.STA_IF)

In [94]:
sta_if.active(True)

In [95]:
print(sta_if.scan())

[(b'raspi-poste1', b"\xb8'\xeb\xb1=u", 1, -40, 3, False), (b'Fablab-inspirateur', b'\xb0\x98+\xcb\xc9n', 1, -63, 4, False), (b'orange', b'\xb2\x98+\xcb\xc9n', 1, -67, 0, False), (b'LInspirateur', b'\xcc@\xd0\r\xc0\xad', 11, -90, 3, False), (b'HP-Print-1D-Officejet Pro 8620', b'\x94W\xa5\xa0\xaf\x1d', 1, -94, 3, False)]


In [96]:
sta_if.connect("raspi-poste1","Burkert67")

In [143]:
Idents=[b"560387",b"2,"b"3",b"4",b"5"]

In [144]:
print(len(Idents))

4


# Client MQTT

In [145]:
from umqtt.simple import MQTTClient

In [146]:
client = MQTTClient('esp32_01','10.3.141.1')
client.connect()

In [216]:
def sub_cb(topic,payload):
    
    global Leds
    global Idents
    off = (0,0,0,0)
    on = (100,100,100,100)
    size = len(Idents)
    if topic==b"off":
        for i in range(size):
            Leds[i]=off
    for i in range(size):
        if topic==Idents[i]:
            print(topic,payload,i,Leds[i])
            Leds[i]=on
    utime.sleep_ms(100)
    Leds.write()

In [217]:
client.set_callback(sub_cb)

In [218]:
for i in range(len(Idents)):
    client.subscribe(Idents[i])
client.subscribe(b"off")


In [219]:
client.publish("esp32_01","ok")

In [ ]:
while 1 :
    client.check_msg()

b'560387' b'1' 0 (0, 0, 0, 0)
......

# 